# Quick Start: Context Engineering Dashboard

Get up and running in 5 cells. This notebook shows how to:

1. Build a context trace manually
2. Visualize it in your notebook
3. Capture a **live OpenAI call** automatically
4. Use **ContextResource** for RAG document pools
5. Compare two context strategies with a Sankey diff

In [ ]:
# Setup: works with pip install OR local development
import sys
from pathlib import Path

# Try importing the package - if it fails, add parent directory to path
try:
    import context_engineering_dashboard
    print(f"Using installed package: {context_engineering_dashboard.__file__}")
except ImportError:
    # Running from local clone - add parent directory to path
    repo_root = Path().resolve().parent
    if repo_root not in sys.path:
        sys.path.insert(0, str(repo_root))
    print(f"Using local development: {repo_root}")

# Installation options (run one if package not found):
# Option 1: Install from GitHub release
# !pip install git+https://github.com/cp71-dlai/context-engineering-dashboard.git@v0.1.0

# Option 2: Install locally for development (from repo root)
# !pip install -e ".[dev,all]"

## 1 | Build a trace by hand

A **ContextTrace** is the core data structure. It holds a list of
**ContextComponents** (system prompt, user message, RAG docs, etc.)
and the model's context-window limit.

In [ ]:
from context_engineering_dashboard import (
    ComponentType,
    ContextComponent,
    ContextTrace,
    ContextBuilder,
)

components = [
    ContextComponent("sys",  ComponentType.SYSTEM_PROMPT, "You are a helpful coding assistant.", token_count=500),
    ContextComponent("rag1", ComponentType.RAG,  "ChromaDB stores embeddings for semantic search.", token_count=4200, metadata={"score": 0.93}),
    ContextComponent("rag2", ComponentType.RAG,  "Collections group related documents together.", token_count=2800, metadata={"score": 0.85}),
    ContextComponent("hist", ComponentType.CHAT_HISTORY,  "User previously asked about installation.", token_count=1100),
    ContextComponent("user", ComponentType.USER_MESSAGE,  "How do I query a Chroma collection?", token_count=350),
]

trace = ContextTrace(
    context_limit=128_000,
    components=components,
    total_tokens=sum(c.token_count for c in components),
)

print(f"Tokens: {trace.total_tokens:,} / {trace.context_limit:,}  ({trace.utilization:.1f}% used)")

## 2 | Visualize the context window

`ContextBuilder` renders an interactive HTML widget right inside the notebook.
Each colored block represents one component, sized proportionally to its token count.

In [ ]:
# Visualize the trace -- hover blocks for details, click to view content
ContextBuilder(trace=trace)

## 3 | Trace a live OpenAI call

Wrap any `openai` call in `trace_openai()`. The tracer captures messages,
token usage, latency, and the response -- then builds the trace for you.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # reads OPENAI_API_KEY from .env

In [ ]:
from openai import OpenAI
from context_engineering_dashboard import trace_openai

client = OpenAI()  # uses OPENAI_API_KEY from environment

with trace_openai() as tracer:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a concise technical writer."},
            {"role": "user",   "content": "Explain what context engineering is and why it matters for LLM applications. Keep it to 3 sentences."},
        ],
        temperature=0.7,
    )

print("Response:", response.choices[0].message.content)
print()

openai_trace = tracer.result
print(f"Prompt tokens: {openai_trace.trace.usage['prompt_tokens']}")
print(f"Completion tokens: {openai_trace.trace.usage['completion_tokens']}")
print(f"Latency: {openai_trace.trace.latency_ms:.0f} ms")

In [ ]:
# Visualize the captured trace -- click components to view, click text to edit
ContextBuilder(trace=openai_trace)

## 4 | ContextResource for RAG document pools

**ContextResource** represents a pool of items (RAG documents, examples, tools, etc.)
that can be selected for inclusion in the context window. Use it to manage
what content is available vs. what actually goes into the LLM call.

In [ ]:
from context_engineering_dashboard import ContextResource, ResourceType

# Create a resource pool from a list of documents
rag_docs = ContextResource.from_items(
    items=[
        {"id": "doc_1", "content": "ChromaDB is an open-source embedding database for AI applications.", "score": 0.95},
        {"id": "doc_2", "content": "Collections in Chroma store documents with their embeddings.", "score": 0.88},
        {"id": "doc_3", "content": "Query with collection.query(query_texts=['...'], n_results=10).", "score": 0.82},
        {"id": "doc_4", "content": "Metadata filtering: use where={'field': 'value'} in queries.", "score": 0.75},
        {"id": "doc_5", "content": "Chroma supports persistent storage with PersistentClient.", "score": 0.70},
    ],
    resource_type=ResourceType.RAG,
    name="Documentation",
)

# Select the top 3 documents for inclusion
rag_docs.select(["doc_1", "doc_2", "doc_3"])

print(f"Resource: {rag_docs.name}")
print(f"Total items: {len(rag_docs.items)}")
print(f"Selected: {len(rag_docs.selected_ids)}")
print(f"Selected tokens: {rag_docs.total_selected_tokens}")

In [ ]:
# Convert selected items to ContextComponents for the trace
rag_components = rag_docs.to_components()

for comp in rag_components:
    print(f"  {comp.id}: {comp.token_count} tokens, type={comp.type.value}")

In [ ]:
# Visualize with resources panel showing available vs. selected
# The left panel shows ALL items; the right shows what's in the context
ContextBuilder(trace=trace, resources=[rag_docs])

## 5 | Compare two strategies with a Sankey diff

Imagine you refactored a prompt: trimmed chat history and dropped a RAG doc.
`ContextDiff` shows token flow between the two versions.

In [ ]:
from context_engineering_dashboard import ContextDiff

# "Before" -- verbose prompt
before = ContextTrace(
    context_limit=128_000,
    components=[
        ContextComponent("sys",  ComponentType.SYSTEM_PROMPT, "...", token_count=3000),
        ContextComponent("hist", ComponentType.CHAT_HISTORY,  "...", token_count=18000),
        ContextComponent("rag1", ComponentType.RAG,  "...", token_count=12000),
        ContextComponent("rag2", ComponentType.RAG,  "...", token_count=5000),
        ContextComponent("user", ComponentType.USER_MESSAGE,  "...", token_count=400),
    ],
    total_tokens=38400,
)

# "After" -- compacted prompt (summarized history, dropped low-score doc)
after = ContextTrace(
    context_limit=128_000,
    components=[
        ContextComponent("sys",  ComponentType.SYSTEM_PROMPT, "...", token_count=3000),
        ContextComponent("hist", ComponentType.CHAT_HISTORY,  "...", token_count=6000),
        ContextComponent("rag1", ComponentType.RAG,  "...", token_count=12000),
        ContextComponent("user", ComponentType.USER_MESSAGE,  "...", token_count=400),
    ],
    total_tokens=21400,
)

diff = ContextDiff(before=before, after=after, before_label="Verbose", after_label="Compacted")
diff.sankey()

In [ ]:
diff.summary()

## 6 | Save & reload traces

Traces serialize to JSON for reproducibility and sharing.

In [ ]:
openai_trace.to_json("quick_start_trace.json")

reloaded = ContextTrace.from_json("quick_start_trace.json")
print(f"Reloaded: {len(reloaded.components)} components, {reloaded.total_tokens:,} tokens")

# Clean up
import os
os.remove("quick_start_trace.json")

---

**Next steps:** See `deep_dive.ipynb` for Chroma integration with ContextResource,
multi-query comparisons, and advanced resource management.